## `datetime` --- 基本的な日付型および時間型
今回は、`datetime` モジュールについて簡単に見ていきます。  
`datetime` モジュールは、日付や時刻を操作するためのクラスを提供しています。

### タイムゾーン変換について Aware なのか Naive なのか
`datetime`モジュールに関わらず、`time`モジュールや`date`モジュールにも、  
**awareオブジェクトと、naiveオブジェクトというものがあります。**  
  
aware オブジェクトは、(国際的な)タイムゾーン情報を相対的に持っており(つまり時刻や時間の比較ができる)、一方、  
naive オブジェクトは、相対的なタイムゾーン情報は持っていないものです。  
(あるプログラム内の数字がメートルを表わしているのか、マイルなのか、  
それとも質量なのかがプログラムによって異なるように、naive オブジェクトが協定世界時 (UTC) なのか、  
現地時間なのか、それとも他のタイムゾーンなのかはそのプログラムに依存します。)

### Aware なのか Naive なのか判断するためには、
**`1.d.tzinfo が None でない`**  
**`2.d.tzinfo.utcoffset(d) が None を返さない`**  
これら両方を満たすとき、オブジェクト `d`は Awareと判断されます。  
どちらか一方を満たさない場合、オブジェクト `d`は Naiveと判断されます。

In [1]:
from datetime import datetime, timedelta, timezone

In [2]:
JST = timezone(timedelta(hours=9))

In [3]:
now = datetime.now(tz=JST)

In [4]:
print(now)

2021-07-08 18:53:24.855572+09:00


In [5]:
now.tzinfo

datetime.timezone(datetime.timedelta(seconds=32400))

In [6]:
now.tzinfo.utcoffset(now)

datetime.timedelta(seconds=32400)

**つまりこの場合、`now`は `aware` オブジェクトと判断できる**

In [7]:
now = datetime.utcnow()

In [8]:
print(now)

2021-07-08 09:53:30.624943


In [9]:
now.tzinfo

In [10]:
now.tzinfo.utcoffset(now)

AttributeError: 'NoneType' object has no attribute 'utcoffset'

**`now`は、`naive`オブジェクト**  
### `aware`オブジェクトと、`naive`オブジェクトは比較できない

In [11]:
now_aware1 = datetime.now(tz=JST)

In [12]:
now_aware2 = datetime.now(tz=JST)

In [13]:
now_naive1 = datetime.utcnow()

In [14]:
now_naive2 = datetime.utcnow()

In [15]:
print(now_aware1, '\n' ,now_aware2)

2021-07-08 18:53:34.517862+09:00 
 2021-07-08 18:53:35.278521+09:00


In [16]:
print(now_naive1,'\n',now_naive2)

2021-07-08 09:53:35.895151 
 2021-07-08 09:53:36.572902


In [17]:
now_aware1 > now_aware2

False

In [18]:
now_aware1 <= now_aware2

True

In [19]:
now_aware1 > now_naive1

TypeError: can't compare offset-naive and offset-aware datetimes

In [20]:
now_naive1 > now_naive2

False

In [21]:
now_naive1 <= now_naive2

True

**`aware`オブジェクトは、タイムゾーンをもつため、例えば日本標準時刻でシステム設計している場合は、**  
**インタフェースから取得された日時は、場合によって日本標準時刻に合わせる必要があります**

### `timedelta` オブジェクト
`class datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)`  
全ての引数がオプションで、デフォルト値は 0 です。 引数は整数、浮動小数点数でもよく、正でも負でもかまいません。  
  
`days`, `seconds`, `microseconds` だけが内部的に保持されます。 引数は以下のようにして変換されます:  
  
・1 ミリ秒は 1000 マイクロ秒に変換されます  
・1 分は 60 秒に変換されます  
・1 時間は 3600 秒に変換されます  
・1 週間は 7 日に変換されます  
  
さらに、値が一意に表されるように `days`, `seconds`, `microseconds` が以下のように正規化されます  
  
`0 <= microseconds < 1000000`  
`0 <= seconds < 3600*24 (一日中の秒数)`  
`-999999999 <= days <= 999999999`  

In [22]:
from datetime import timedelta

In [23]:
delta = timedelta(
    days=50,
    seconds=27,
    microseconds=10,
    milliseconds=29000,
    minutes=5,
    hours=8,
    weeks=2)

In [24]:
delta

datetime.timedelta(days=64, seconds=29156, microseconds=10)

**`delta`には合計された値が返る**  
指定された範囲の外側になった場合には、 `OverflowError` が返えされます。

In [25]:
delta = timedelta(
    days=50,
    seconds=27,
    microseconds=10,
    milliseconds=1000000000000000000000000000000000, # 指定範囲外
    minutes=5,
    hours=8,
    weeks=2)

OverflowError: Python int too large to convert to C int

`timedelta.min`  
最小の値を表す `timedelta` オブジェクトで、 `timedelta(-999999999)` です  
  
`timedelta.max`  
最大の値を表す `timedelta` オブジェクトで、 `timedelta(days=999999999, hours=23, minutes=59, seconds=59, microseconds=999999) `です  
  
`timedelta.resolution`
`timedelta` オブジェクトが等しくならない最小の時間差で、 `timedelta(microseconds=1)` です。

**`==` および `!=` の比較は、比較されているオブジェクトの型が何であれ、 常に bool を返します**

In [26]:
delta1 = timedelta(seconds=57)

In [27]:
delta2 = timedelta(hours=25, seconds=2)

In [28]:
delta1 == delta2

False

In [29]:
delta1 != delta2

True

In [30]:
delta1

datetime.timedelta(seconds=57)

In [31]:
delta2

datetime.timedelta(days=1, seconds=3602)

In [32]:
delta2 > delta1

True

**より具体的な使い方の例...**

In [33]:
year = timedelta(days=365)

In [34]:
ten_years = 10 * year

In [35]:
ten_years

datetime.timedelta(days=3650)

In [36]:
ten_years.days // 365

10

In [37]:
nine_years = ten_years - year

In [38]:
nine_years

datetime.timedelta(days=3285)

In [39]:
three_years = nine_years // 3

In [40]:
three_years, three_years.days // 365

(datetime.timedelta(days=1095), 3)

### `date` オブジェクト

In [41]:
from datetime import date

In [42]:
target_date = '2021-07-08'

In [43]:
print(target_date, type(target_date))

2021-07-08 <class 'str'>


In [45]:
date.fromisoformat(target_date)

datetime.date(2021, 7, 8)

**`YYYY-MM-DD` という書式のみをサポートしています**  
`date.year`  
両端値を含む `MINYEAR` から `MAXYEAR` までの値です  
`date.month`  
両端値を含む 1 から 12 までの値です  
`date.day`  
1 から与えられた月と年における日数までの値です  

In [47]:
d = date(2021, 7, 8)

In [48]:
d.year

2021

In [49]:
d.month

7

In [50]:
d.day

8

In [51]:
d = date(2021, 7, 32)

ValueError: day is out of range for month

In [52]:
d = date(20215, 7, 31)

ValueError: year 20215 is out of range

In [54]:
d = date(2021, 13, 31)

ValueError: month must be in 1..12

In [55]:
d = date(9999, 12, 31)

In [56]:
d = date(-9999, 12, 31)

ValueError: year -9999 is out of range

`date.weekday()`  
月曜日を 0、日曜日を 6 として、曜日を整数で返します。  
例えば、 `date(2002, 12, 4).weekday() == 2` であり、水曜日を示します
  
`date.isoweekday()`  
月曜日を 1,日曜日を 7 として、曜日を整数で返します。  
例えば、 `date(2002, 12, 4).isoweekday() == 3` であり、水曜日を示します。   
  
`date.isocalendar()`
`year`、`week`、`weekday` の3つで構成された `named tuple` を返します。
  
ISO 暦はグレゴリオ暦の変種として広く用いられています。  
ISO 年は完全な週が 52 週または 53 週あり、週は月曜から始まって日曜に終わります。  
ISO 年でのある年における最初の週は、その年の木曜日を含む最初の (グレゴリオ暦での) 週となります。  
この週は週番号 1 と呼ばれ、この木曜日での ISO 年はグレゴリオ暦における年と等しくなります。  
  
例えば、2004 年は木曜日から始まるため、ISO 年の最初の週は 2003 年 12 月 29 日、月曜日から始まり、2004 年 1 月 4 日、日曜日に終わります  
  
**ISO8601フォーマット**  
`yyyy-MM-ddTHH:mm:ss`のような形で日時を表現します。

In [59]:
### 水曜日
date(2002, 12, 4).weekday()

2

In [60]:
### 水曜日
date(2002, 12, 4).isoweekday()

3

In [61]:
### 木曜日
date(2021, 7, 8).weekday()

3

In [62]:
### 木曜日
date(2021, 7, 8).isoweekday()

4

In [63]:
date(2021, 7, 8).isocalendar()

(2021, 27, 4)

`date.isoformat()`
日付を ISO 8601 書式の `YYYY-MM-DD` で表した文字列を返します

In [64]:
date(2021, 7, 8).isoformat()

'2021-07-08'

In [65]:
### 例えば...イベントまでの日数をカウントする場合
import time
from datetime import date

In [66]:
today = date.today()

In [67]:
today

datetime.date(2021, 7, 8)

In [69]:
my_birthday = date(today.year, 2, 24)

In [70]:
### 今年すでに誕生日を迎えてしまっていたら、翌年の誕生日を考慮する
if my_birthday < today:
    my_birthday = my_birthday.replace(year=today.year + 1)

In [71]:
my_birthday

datetime.date(2022, 2, 24)

In [72]:
time_to_birthday = abs(my_birthday - today)

In [73]:
### 次の誕生日まで...
time_to_birthday

datetime.timedelta(days=231)

### 日付操作のいろいろ

**[困ったときのタイムゾーン確認サイト](https://www.jisakeisan.com/#list)**

In [74]:
from datetime import date

In [75]:
today = date.today()

In [76]:
print(today)

2021-07-08


In [77]:
print('year: {}, month: {}, day: {}'.format(today.year, today.month, today.day))

year: 2021, month: 7, day: 8


In [78]:
from datetime import date

In [79]:
WEEKDAY = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']

In [82]:
print('本日は、{}です'.format(WEEKDAY[today.weekday()])) # isoは0から6

本日は、木曜日です


In [83]:
print('本日は、{}です'.format(WEEKDAY[today.isoweekday()-1])) # isoは１から７

本日は、木曜日です


In [84]:
past_date = date(2020, 7, 8)

In [85]:
print(past_date.strftime('%Y年%-m月%-d日{}'.format(WEEKDAY[past_date.weekday()])))

2020年7月8日水曜日


### strftime() と strptime() の振る舞い  
`strftime` : オブジェクトを与えられた書式に従って文字列に変換する  
`strptime` : 指定された対応する書式で文字列を構文解析して `datetime` オブジェクトにする  

In [87]:
date1 = date(2021, 7, 8)
date2 = date.fromisoformat('2021-06-08')

if date1 < date2:
    print('{}の方が、{}よりも、日付が古いです'.format(date1, date2))
elif date1 == date2:
    print('二つは、同じ日付です')
else:
    print('{}の方が、{}よりも、日付が古いです'.format(date2, date1))

2021-06-08の方が、2021-07-08よりも、日付が古いです


In [88]:
from datetime import datetime, time

In [89]:
now = datetime.now().time()
print('現在時刻は、{}時{}分{}.{}秒　です'.format(now.hour, now.minute, now.second, now.microsecond)) 

現在時刻は、19時25分57.254732秒　です


In [90]:
# 現在時刻を取得する
now = datetime.now().time()

In [91]:
# isoフォーマットで出力(マイクロ秒あり)
print(now.isoformat()) 

19:26:20.819020


In [92]:
# isoフォーマットで出力(マイクロ秒なし)
print(now.isoformat(timespec='seconds'))

19:26:20


In [93]:
# 任意の形式で出力
print(now.strftime('%-H時%-M分%-S.%f秒'))

19時26分20.819020秒


代表的な、指定子...  
**`%H` : 0埋めした10進数で表記した時 (24時間表記)。**  
**`%M` : 0埋めした10進数で表記した分。**  
**`%S`: 0埋めした10進数で表記した秒。**  
**`%f` :10進数で表記したマイクロ秒 (左側から0埋めされます)。**

In [94]:
from datetime import datetime

In [95]:
# ローカルの現在時刻を取得する(日本現地時間)
now = datetime.today()
print(now)

2021-07-08 19:29:55.689361


In [96]:
# UTC現在時刻を取得する(協定世界時、イギリスの現地時間)
utc_now = datetime.utcnow()
print(utc_now)

2021-07-08 10:30:00.717811


In [110]:
# Unixタイムスタンプから生成する
unix_time = datetime.fromtimestamp(1625719335)
print(unix_time) # 2021-07-08 13:42:15

2021-07-08 13:42:15


In [111]:
from datetime import datetime, timedelta

# 現在時間を取得する
now = datetime.today()
print(now)

2021-07-08 19:32:27.235236


In [112]:
# 日にちを1日進める
print(now + timedelta(days=1))

2021-07-09 19:32:27.235236


In [113]:
# 日にちを1日遅らせる
print(now - timedelta(days=1))

2021-07-07 19:32:27.235236


In [114]:
# 1時間進める
print(now + timedelta(hours=1))

2021-07-08 20:32:27.235236


### 月年単位での時間操作

In [115]:
from dateutil import parser, relativedelta

In [116]:
date1 = '2021-07-08 19:00:00'

In [117]:
date1_parsed = parser.parse(date1)

In [118]:
# 1月加算
# 2021-08-08 19:00:00
print(date1_parsed + relativedelta.relativedelta(months=1))

2021-08-08 19:00:00


In [119]:
# 1月減算
# 2021-06-08 19:00:00
print(date1_parsed - relativedelta.relativedelta(months=1))

2021-06-08 19:00:00


In [120]:
# 1年加算
# 2022-07-08 19:00:00
print(date1_parsed + relativedelta.relativedelta(years=1))

2022-07-08 19:00:00


In [121]:
# 1年減算
# 2020-07-08 19:00:00
print(date1_parsed - relativedelta.relativedelta(years=1))

2020-07-08 19:00:00


## 以上となります...！
**次回は、**  
**calendar 、collections モジュールを見ていきます...！**